In [ ]:
%pip install huggingface-hub

In [ ]:
from huggingface_hub import HfApi, hf_hub_url, login
import requests
import os
import boto3
import botocore

In [ ]:
login_token = os.environ.get('huggingface_token')
login(token=login_token)
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

In [ ]:

# 2) boto3 client for your MinIO/S3
session = boto3.session.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)
s3 = session.resource(
    "s3",
    endpoint_url=endpoint_url,
    region_name=region_name,
    config=botocore.client.Config(signature_version="s3v4"),
)
bucket = s3.Bucket(bucket_name)

In [ ]:
# 3) which repo & prefix you’ll stream into
model_prefix = "Qwen"
model_name   = "Qwen2-VL-7B-Instruct"
repo_id      = f"{model_prefix}/{model_name}"
s3_prefix    = f"{model_name}/"   # all files go under this key prefix

In [ ]:
# 4) list every file in the HF repo
api = HfApi()
files = api.list_repo_files(repo_id, token=login_token)

In [ ]:
for fn in files:
    if fn.endswith("/"):
        continue

    print(f"→ {fn}")
    url = hf_hub_url(repo_id=repo_id, filename=fn)
    # append ?token= to get a presigned-like URL for private models
    url_with_token = f"{url}?token={login_token}"
    resp = requests.get(url_with_token, stream=True)
    resp.raise_for_status()
    resp.raw.decode_content = True

    # stream directly into your S3/MinIO bucket
    bucket.upload_fileobj(
        Fileobj=resp.raw,
        Key=s3_prefix + fn
    )